In [1]:
from greet import greet
greet('freddy')

'Hi there, freddy'

In [2]:
animals = ['dog', 'cat', 'tiger']
pairs = enumerate(animals)
reverse = {key: index for (index, key) in pairs}
reverse

{'cat': 1, 'dog': 0, 'tiger': 2}

In [3]:
animals[0] = 'canine'
animals

['canine', 'cat', 'tiger']

In [4]:
nums = ['one','two','three','four','five','six','seven','eight','nine','ten']
odds = nums[::2]
evens = nums[1::2]

print('evens', evens)
print('odds', odds)

backwards = nums[::-1]
print('backwards', backwards)

evens ['two', 'four', 'six', 'eight', 'ten']
odds ['one', 'three', 'five', 'seven', 'nine']
backwards ['ten', 'nine', 'eight', 'seven', 'six', 'five', 'four', 'three', 'two', 'one']


In [5]:
# an enumeration cannot be reused
forward = {index: key for (index, key) in pairs}
forward  

{}

In [6]:
# better to inline the enumerate
forward = {index: key for (index, key) in enumerate(animals)}
print('forward', forward)
reverse = {key: index for (index, key) in enumerate(animals)}
print('reverse', reverse)

forward {0: 'canine', 1: 'cat', 2: 'tiger'}
reverse {'canine': 0, 'cat': 1, 'tiger': 2}


In [7]:
evens_squared = [a**2 for a in range(2,10,2)]
print(evens_squared)

[4, 16, 36, 64]


In [8]:
pets = ['dog','cat', 'parrot']
counts = [2,3,0]
pet_lookup = {pet:count for (pet, count) in zip(pets, counts) if count > 0}
print(pet_lookup)

{'dog': 2, 'cat': 3}


In [9]:
pc = zip(pets, counts)
for l in pc:
    print(l)

('dog', 2)
('cat', 3)
('parrot', 0)


In [10]:
for l in pc:
    print(l)

### Note   
Both zip and enumerate produce single use iterators.
Inline is always better...

In [11]:
def create_greeter(greeting = 'Hi'):
    count = 0
    def greet(name):
        nonlocal count
        count += 1
        return '%s %s %d' % (greeting, name, count)
    
    return greet
    
formal_greeter = create_greeter('Hello there,')
print(formal_greeter('Fred'))
print(formal_greeter('Ted'))

informal_greeter = create_greeter('Hey')
print(informal_greeter('Fred'))
print(informal_greeter('Ted'))



Hello there, Fred 1
Hello there, Ted 2
Hey Fred 1
Hey Ted 2


In [12]:
def create_greeter(greeting = 'Hi'):
    context = {'count':0}
    def greet(name):
        context['count'] += 1
        return '%s %s %d' % (greeting, name, context['count'])
    
    return greet
    
formal_greeter = create_greeter('Hello there,')
print(formal_greeter('Fred'))
print(formal_greeter('Ted'))

informal_greeter = create_greeter('Hey')
print(informal_greeter('Fred'))
print(informal_greeter('Ted'))
print(formal_greeter('Red'))
print(informal_greeter('Red'))



Hello there, Fred 1
Hello there, Ted 2
Hey Fred 1
Hey Ted 2
Hello there, Red 3
Hey Red 3


### Note
Assignment to a closure scope variable requires use of the nonlocal keyword. Mutating the contents of an object does not.

In [13]:
def add_with_side_effects_lost1(a,b):
    temp_a = a
    a = a + 10
    return temp_a + b

first = 4
second = 6
print('sum of 4 and 6', add_with_side_effects_lost1(first, second))
print('first', first)

sum of 4 and 6 10
first 4


In [14]:
def add_with_side_effects(numbers):
    temp_first = numbers[0]
    numbers[0] += 10
    return temp_first + numbers[1]

nmbrs = [4,6]
print('sum of 4 and 6', add_with_side_effects(nmbrs))
print('nmbrs', nmbrs)

sum of 4 and 6 10
nmbrs [14, 6]


In [15]:
def add_with_side_effects_lost2(numbers):
    temp_sum = numbers[0] + numbers[1]
    numbers = [99, 199]
    return temp_sum

nmbrs = [4,6]
print('sum of 4 and 6', add_with_side_effects_lost2(nmbrs))
print('nmbrs', nmbrs)

sum of 4 and 6 10
nmbrs [4, 6]


### Passing function arguments 
Behaves like passing a copy of the reference.
Assignments are lost. Changes to the contents of a complex type bubble out of the function.

In [20]:
def add(name, number, *values):
    return sum(values)

add('fred', 12, 1,2,3)

6

In [23]:
numbers = [1,2,3]
add('fred', 12, *numbers)

6

In [26]:
[a,b,c] = numbers
print(b)

2
